In [2]:
import numpy as np
import cv2
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [3]:
directory = 'TCGA-XF-AAN2-01Z-00-DX1.EB523A3A-0DE0-4FFC-9FE7-CF4FB2FB36CF.svs'
files = os.listdir(directory)
files

['100001_12001_4000_4000_0.2527_1-features.csv',
 '100001_16001_4000_4000_0.2527_1-features.csv',
 '100001_1_4000_4000_0.2527_1-features.csv',
 '100001_20001_4000_4000_0.2527_1-features.csv',
 '100001_24001_4000_4000_0.2527_1-features.csv',
 '100001_28001_4000_4000_0.2527_1-features.csv',
 '100001_32001_4000_4000_0.2527_1-features.csv',
 '100001_36001_4000_4000_0.2527_1-features.csv',
 '100001_40001_4000_4000_0.2527_1-features.csv',
 '100001_4001_4000_4000_0.2527_1-features.csv',
 '100001_44001_4000_4000_0.2527_1-features.csv',
 '100001_48001_4000_4000_0.2527_1-features.csv',
 '100001_52001_4000_4000_0.2527_1-features.csv',
 '100001_56001_4000_4000_0.2527_1-features.csv',
 '100001_60001_4000_4000_0.2527_1-features.csv',
 '100001_64001_4000_4000_0.2527_1-features.csv',
 '100001_68001_4000_4000_0.2527_1-features.csv',
 '100001_72001_4000_4000_0.2527_1-features.csv',
 '100001_76001_4000_4000_0.2527_1-features.csv',
 '100001_80001_4000_4000_0.2527_1-features.csv',
 '100001_8001_4000_4000_0

In [4]:
# Obtain dimensions of the WSI image
x_coords = []
y_coords = []
x_widths = []
y_heights = []
for file in files:
    parts = file.split('_')
    x_coords.append(int(parts[0]))
    y_coords.append(int(parts[1]))
    x_widths.append(int(parts[2]))
    y_heights.append(int(parts[3]))

# X_Size
# Obtain the maximum x coordinate and its corresponding width
x_max = max(x_coords)
x_min = min(x_coords)
x_max_index = x_coords.index(x_max)
x_max_width = x_widths[x_max_index]
# X_Size = x_max - x_min + x_max_width
X_Size = x_max - x_min + x_max_width

# Y_Size
# Obtain the maximum y coordinate and its corresponding height
y_max = max(y_coords)
y_min = min(y_coords)
y_max_index = y_coords.index(y_max)
y_max_height = y_heights[y_max_index]
# Y_Size = y_max - y_min + y_max_height
Y_Size = y_max - y_min + y_max_height

print("X_Size: ", X_Size)
print("Y_Size: ", Y_Size)

X_Size:  127481
Y_Size:  88000


In [5]:
global_mask = np.zeros((Y_Size, X_Size), dtype=np.uint8)

# Iterate through the files, creating each local mask and adding it to the global mask
for file in files:
    parts = file.split('_')
    x_coord = int(parts[0]) - x_min
    y_coord = int(parts[1]) - y_min
    x_width = int(parts[2])
    y_height = int(parts[3])
    
    # Create a local mask for the current file
    local_mask = np.zeros((y_height, x_width), dtype=np.uint8)
    
    # Parse Polygon coordinates
    with open(os.path.join(directory, file), 'r') as f:
        df = pd.read_csv(f)
        if len(df) > 0:
            for _, row in df.iterrows():
                # Parse polygon coordinates
                coords = np.array(
                    row['Polygon'].strip('[]').split(':'), dtype=np.float32
                ).reshape(-1, 2)

                # Calculate Centroid of the polygon
                centroid = np.mean(coords, axis=0).astype(int)
                cx, cy = centroid

                # Ensure the centroid is within the bounds of the local mask
                if 0 <= cx < x_width and 0 <= cy < y_height:
                    local_mask[cy, cx] = 1
        
    # Update the global mask with the local mask
    global_mask[y_coord:y_coord + y_height, x_coord:x_coord + x_width] = np.maximum(
        global_mask[y_coord:y_coord + y_height, x_coord:x_coord + x_width], local_mask
    )

# Save the global mask as a .png file
output_path = os.path.join(directory, 'global_mask.png')
cv2.imwrite(output_path, global_mask * 255)
print(f"Global mask saved to {output_path}")    

MemoryError: Unable to allocate 10.4 GiB for an array with shape (88000, 127481) and data type uint8